In [10]:
import sys
import re
import numpy as np
import pandas as pd
import matplotlib as plt
from nltk.tokenize import RegexpTokenizer

# np.set_printoptions(edgeitems=3)
# np.core.arrayprint._line_width = 80

In [3]:
path = 'data.txt'
data = open(path, 'r', encoding='utf-8').read().lower()
print('length of the corpus is: ', len(data))

length of the corpus is:  581888


In [16]:
# splitting the corpus into words

cleaned = re.sub(r'\W+', ' ', data).lower()
tokens = tokenizer.tokenize(cleaned)
print('Length of tokens: ' + str(len(tokens)))

# This includes all full stops and all the punctuation. We only need the words...
# tokenizer = RegexpTokenizer(r'\w+|\$[\d\.]+|\S+')
# words = tokenizer.tokenize(data)
# len(words)

Length of tokens: 109226


In [8]:
# Finding all the unique words

unique_words = np.unique(words)
len(unique_words)

9358

In [17]:
# Feature Engineering - converting the text into numerical values

train_len = 5
text_sequences = []
for i in range(train_len,len(tokens)):
    seq = tokens[i-train_len:i]
    text_sequences.append(seq)


In [20]:
text_sequences[0:20]

[['project', 'gutenberg', 's', 'the', 'adventures'],
 ['gutenberg', 's', 'the', 'adventures', 'of'],
 ['s', 'the', 'adventures', 'of', 'sherlock'],
 ['the', 'adventures', 'of', 'sherlock', 'holmes'],
 ['adventures', 'of', 'sherlock', 'holmes', 'by'],
 ['of', 'sherlock', 'holmes', 'by', 'arthur'],
 ['sherlock', 'holmes', 'by', 'arthur', 'conan'],
 ['holmes', 'by', 'arthur', 'conan', 'doyle'],
 ['by', 'arthur', 'conan', 'doyle', 'this'],
 ['arthur', 'conan', 'doyle', 'this', 'ebook'],
 ['conan', 'doyle', 'this', 'ebook', 'is'],
 ['doyle', 'this', 'ebook', 'is', 'for'],
 ['this', 'ebook', 'is', 'for', 'the'],
 ['ebook', 'is', 'for', 'the', 'use'],
 ['is', 'for', 'the', 'use', 'of'],
 ['for', 'the', 'use', 'of', 'anyone'],
 ['the', 'use', 'of', 'anyone', 'anywhere'],
 ['use', 'of', 'anyone', 'anywhere', 'at'],
 ['of', 'anyone', 'anywhere', 'at', 'no'],
 ['anyone', 'anywhere', 'at', 'no', 'cost']]